In [5]:
from PIL import Image
import numpy as np

In [11]:
def encode_lsb(image_path, text, output_path):
    img = Image.open(image_path)
    img = img.convert("RGB")
    data = np.array(img)
    flat_data = data.flatten()

    # Zamiana tekstu na bity
    bits = ''.join(f"{ord(c):08b}" for c in text)
    bits += '00000000'  # bajt stopu

    if len(bits) > len(flat_data):
        raise ValueError("Za dużo danych do zakodowania w obrazie")

    for i, bit in enumerate(bits):
        flat_data[i] = (flat_data[i] & ~1) | int(bit)

    encoded_img = Image.fromarray(flat_data.reshape(data.shape))
    encoded_img.save(output_path)

In [12]:
def decode_lsb(image_path):
    img = Image.open(image_path)
    data = np.array(img).flatten()

    bits = ""
    for val in data:
        bits += str(val & 1)

        # Co 8 bitów sprawdzamy znak
        if len(bits) % 8 == 0:
            char = chr(int(bits[-8:], 2))
            if char == '\x00':  # bajt stopu
                break
                
    message = ''.join(chr(int(bits[i:i+8], 2)) for i in range(0, len(bits)-8, 8))
    return message

In [14]:
encode_lsb("original.png", "Litwo ojczyzno moja", "encoded.png")
print(decode_lsb("encoded.png"))

Litwo ojczyzno moja


## Pytania

### 1. Czy taki sposób ukrywania informacji w obrazie jest odporny na ataki i próby zniszczenia osadzonej wiadomości?

Nie, metoda LSB (Least Significant Bit) nie jest odporna na ataki ani przypadkowe usunięcie ukrytej informacji. Jest to technika prosta i łatwa do złamania, ponieważ:
- Wiadomość jest wstawiana w sposób deterministyczny (np. od pierwszego piksela w przód),
- Nie stosuje się żadnego szyfrowania ani maskowania danych,
- Dane ukryte są w najmłodszych bitach bajtów obrazu, co czyni je bardzo podatnymi na modyfikacje,
- Zmiany takie jak kompresja, filtracja czy edycja obrazu mogą całkowicie zniszczyć ukrytą wiadomość.

---

### 2. Przykładowe ataki na osadzoną wiadomość

1. **Kompresja stratna (np. JPEG)**  
   - Usuwa drobne różnice w pikselach, przez co niszczy dane w LSB.

2. **Zmiana formatu lub edycja obrazu**  
   - Operacje takie jak zmiana rozdzielczości, jasności, kontrastu lub zastosowanie filtrów mogą nadpisać ukryte dane.

3. **Dodanie szumu**  
   - Nawet drobne zaburzenia mogą zniszczyć zakodowane bity.

4. **Atak różnicowy (diff attack)**  
   - Porównanie obrazu zmodyfikowanego z oryginałem pozwala wykryć zmiany w LSB.

---

### 3. Jaki jest maksymalny rozmiar wiadomości możliwy do ukrycia?

Zakładając, że każdy najmłodszy bit bajtu (LSB) może przechować jeden bit wiadomości:

- W obrazie RGB: **1 piksel = 3 bajty** (R, G, B), co daje **3 bity możliwe do wykorzystania** na wiadomość.
- Aby zakodować **1 znak (8 bitów)**, potrzeba **8 bajtów**, czyli około **2.67 pikseli**.

**Wzór na maksymalny rozmiar wiadomości w znakach:**
- maksymalna liczba znaków = (szerokość × wysokość × 3) // 8 - 1
- `// 8` – bo 8 bitów = 1 znak,
- `-1` – rezerwujemy ostatni bajt na bajt stopu (`\x00`), który sygnalizuje koniec wiadomości.